In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB
from tqdm import tqdm

In [2]:
with open('./clusters.json', 'r') as file:
    clusters = json.load(file)
clusters
print(*map(lambda x: len(x), clusters))
print(len(clusters))

5 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
25


In [3]:
c = pd.read_json('../cleandata/dist_mat.json')
t = pd.read_json('../cleandata/time_mat.json')
locdem = pd.read_excel('../cleandata/locdem.xlsx')
#8 - 125,11 - 126
c = c.loc[:124, :124].to_numpy()
t = t.loc[:124, :124].to_numpy()
q = locdem['Number of pallets']
q = np.array([0]+list(q))

def get(i):
    if i == 125:
        return 8
    if i == 126:
        return 11
    return i

def time(i,j):
    i,j = get(i), get(j)
    return t[i,j]

def cost(i,j):
    if (i,j) in [(126,11), (11,126), (8,125), (125,8)]:
        return 1e+8
    i,j = get(i), get(j)
    return c[i,j]

In [4]:
Q = 9
iter_count = 0
primary_models = []
Ls = []

for cluster in clusters:
    Neighbours = list(sorted(cluster))
    if 8 in Neighbours:
        Neighbours.append(124)
    if 11 in Neighbours:
        Neighbours.append(125)
        
    all_nodes = [0] + Neighbours.copy()
    v = len(all_nodes)
    print(len(cluster) - v)
    V = list(range(v))
    N = V.copy()
    N.remove(0)
    
    def Del(i):
        F = V.copy()
        F.remove(i)
        return F
    
    m = gb.Model()
    x = m.addVars(v,v, vtype=GRB.BINARY, name='x')
    m.setObjective(sum([cost(all_nodes[i], all_nodes[j]) * x[i,j] for i,j in product(V,V)]))
    for i in N:
        m.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
    for j in N:
        m.addConstr(sum([x[i,j] for i in Del(j)]) == 1)
        
    l = m.addVar(lb=0.0, vtype=GRB.INTEGER, name='L')
    Ls.append(l)
    m.addConstr(sum([x[0,j] for j in Del(0)]) == l)
    
    u = m.addVars(V, vtype=GRB.CONTINUOUS, name='u')
    for i,j in filter(lambda tup: tup[0] != tup[1], product(N,N)):
        m.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
    for i in V:
        m.addConstr(u[i] >= q[i])
    
    primary_models.append(m)

-1
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12
-1
-1
-1
-1
-1
-1
-2
-1
-1
-2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [5]:
print(len(primary_models))
for model in tqdm(primary_models):
    model.optimize();

25


  0%|                                                    | 0/25 [00:00<?, ?it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 37 rows, 43 columns and 122 nonzeros
Model fingerprint: 0xd18a4b22
Variable types: 6 continuous, 37 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [9e+03, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 305376.00000
Presolve removed 7 rows and 18 columns
Presolve time: 0.00s
Presolved: 30 rows, 25 columns, 100 nonzeros
Variable types: 5 continuous, 20 integer (20 binary)
Found heuristic solution: objective 162041.00000

Root relaxation: objective 6.756267e+04, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 67562.6667    0    8 162041.000 67562.6667  58.3

 12%|█████▎                                      | 3/25 [00:00<00:01, 21.77it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 37 rows, 43 columns and 122 nonzeros
Model fingerprint: 0x178ba82d
Variable types: 6 continuous, 37 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e+01, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 439947.00000
Presolve removed 7 rows and 18 columns
Presolve time: 0.00s
Presolved: 30 rows, 25 columns, 100 nonzeros
Variable types: 5 continuous, 20 integer (20 binary)
Found heuristic solution: objective 176505.00000

Root relaxation: objective 4.253000e+03, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4253.00000    0    9 176505.000 4253.00000  97.6

...

 80%|██████████████████████████████████▍        | 20/25 [00:01<00:00, 18.33it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 37 rows, 43 columns and 122 nonzeros
Model fingerprint: 0xbc2a497f
Variable types: 6 continuous, 37 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [7e+03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 765676.00000
Presolve removed 7 rows and 18 columns
Presolve time: 0.00s
Presolved: 30 rows, 25 columns, 100 nonzeros
Variable types: 5 continuous, 20 integer (20 binary)
Found heuristic solution: objective 202111.00000

Root relaxation: objective 9.874737e+04, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 98747.3704    0   10 202111.000 98747.3704  51.1

 92%|███████████████████████████████████████▌   | 23/25 [00:01<00:00, 19.40it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 37 rows, 43 columns and 122 nonzeros
Model fingerprint: 0x27063f41
Variable types: 6 continuous, 37 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+03, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 433172.00000
Presolve removed 7 rows and 18 columns
Presolve time: 0.00s
Presolved: 30 rows, 25 columns, 100 nonzeros
Variable types: 5 continuous, 20 integer (20 binary)
Found heuristic solution: objective 178757.00000

Root relaxation: objective 1.479500e+04, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 14795.0000    0    9 178757.000 14795.0000  91.7

100%|███████████████████████████████████████████| 25/25 [00:01<00:00, 16.71it/s]


In [6]:
model_solutions = {}
for i,model in enumerate(primary_models):
    soln = json.loads(model.getJSONSolution())
    model_solutions[i] = soln
json.dump(model_solutions, open('./primary_model_solutions.json', 'w'), indent=4)